In [1]:
from typing import TypedDict, Annotated, List, Union
from dotenv import load_dotenv
import os
from loguru import logger
import operator
import random

In [3]:
from langchain_openai.chat_models import AzureChatOpenAI

In [29]:
load_dotenv("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env", override=True)

True

In [4]:
llm = AzureChatOpenAI(
        deployment_name="gpt-4-turbo-0125",
        model="gpt-4-turbo-0125",
        openai_api_type="azure",
        azure_endpoint=os.environ['AZURE_API_BASE'],
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version=os.environ['AZURE_API_VERSION'],
        max_retries=2,
        temperature=0,
    )

In [5]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-7c47ce35-9f06-43b8-ae20-57d7a7c2f98d-0')

In [6]:
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph


In [7]:
graph = MessageGraph()
graph.add_node("oracle", llm)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")
runnable = graph.compile()

In [8]:
type(runnable)

langgraph.graph.state.CompiledStateGraph

In [9]:
runnable.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
  +--------+   
  | oracle |   
  +--------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [10]:
runnable.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', id='bebd2a98-46c0-4585-bb9e-13ec81727434'),
 AIMessage(content='1 + 1 equals 2.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-ea7f3fb7-28ec-4ebd-b57d-fc5cde72c2b7-0')]

So what did we do here? Let's break it down step by step:

1. First, we initialize our model and a MessageGraph.

2. Next, we add a single node to the graph, called "oracle", which simply calls the model with the given input.

3. We add an edge from this "oracle" node to the special string END. This means that execution will end after current node.

4. We set "oracle" as the entrypoint to the graph.

5. We compile the graph, ensuring that no more modifications to it can be made.

Then, when we execute the graph:


1. LangGraph adds the input message to the internal state, then passes the state to the entrypoint node, "oracle".

2. The "oracle" node executes, invoking the chat model.

3. The chat model returns an AIMessage. LangGraph adds this to the state.

4. Execution progresses to the special END value and outputs the final state.


And as a result, we get a list of two chat messages as output.

## Langgraph - the easy way

> Notes on tutorial by Menlo Park Lab: https://www.youtube.com/watch?v=R8KB-Zcynxc&t=173s

---

### Simple example

In [11]:
def function_1(input_1: str) -> str:
    return input_1 + " Hi "

def function_2(input_2: str) -> str:
    return input_2 + "there:)"

In [12]:
from langgraph.graph import Graph

In [13]:
#define a langchain graph
workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge("node_1", "node_2")

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [14]:
app.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
  +--------+   
  | node_1 |   
  +--------+   
      *        
      *        
      *        
  +--------+   
  | node_2 |   
  +--------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [15]:
app.invoke("hello")

'hello Hi there:)'

In [17]:
input = "hello"
for output in app.stream(input):
    print (output)

{'node_1': 'hello Hi '}
{'node_2': 'hello Hi there:)'}


In [18]:
llm.invoke("hey there").content

'Hello! How can I assist you today?'

### Simple example with LLM call

In [19]:
def function_1(input_1: str) -> str:
    response = llm.invoke(input_1)
    return response.content

def function_2(input_2: str) -> str:
    return "Agent says: " + input_2

In [20]:
workflow = Graph()

workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge("agent", "node_2")

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [21]:
app.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
  +-------+    
  | agent |    
  +-------+    
      *        
      *        
      *        
  +--------+   
  | node_2 |   
  +--------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [22]:
app.invoke("tell me a joke")

"Agent says: Why don't skeletons fight each other?\n\nThey don't have the guts."

In [23]:
input = "tell me a joke"
for output in app.stream(input):
    print (output)

{'agent': "Why don't skeletons fight each other?\n\nThey don't have the guts."}
{'node_2': "Agent says: Why don't skeletons fight each other?\n\nThey don't have the guts."}


### Example 3: Get weather

In [24]:
def city_extractor(input: str) -> str:
    prompt = f"Your task is to extract the city name from the user query. Nothing more, just the name if the city.\
        User query:" + input
    response = llm.invoke(prompt)
    return response.content

In [25]:
def function_2(input_2: str) -> str:
    return "Agent says: " + input_2

In [26]:
workflow = Graph()

workflow.add_node("agent", city_extractor)
workflow.add_node("node_2", function_2)

workflow.add_edge("agent", "node_2")

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [28]:
app.invoke("Whats the weather like in Bangalore?")

'Agent says: Bangalore'

In [31]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper

In [32]:
weather = OpenWeatherMapAPIWrapper()

In [36]:
weather_data = weather.run("Bangalore")

In [37]:
print (weather_data)

In Bangalore, the current weather is as follows:
Detailed status: few clouds
Wind speed: 2.57 m/s, direction: 220°
Humidity: 55%
Temperature: 
  - Current: 30.8°C
  - High: 30.9°C
  - Low: 30.8°C
  - Feels like: 33.29°C
Rain: {}
Heat index: None
Cloud cover: 20%


In [38]:
def get_weather_from_city(city_name: str) -> str:
    
    try:
        weather_data = weather.run(city_name)
    except:
        return f"Weather not found for city: {city_name}"
    return weather_data

In [43]:
workflow = Graph()

workflow.add_node("agent", city_extractor)
workflow.add_node("weather_tool", get_weather_from_city)

workflow.add_edge("agent", "weather_tool")

workflow.set_entry_point("agent")
workflow.set_finish_point("weather_tool")

app = workflow.compile()

In [45]:
print(app.invoke("Whats the weather like in Kolkata?"))

In Kolkata, the current weather is as follows:
Detailed status: haze
Wind speed: 5.14 m/s, direction: 200°
Humidity: 49%
Temperature: 
  - Current: 32.97°C
  - High: 32.97°C
  - Low: 32.97°C
  - Feels like: 35.95°C
Rain: {}
Heat index: None
Cloud cover: 40%
